# Transfer Markt Scraping - Summer Transfers 19/20

Explicar isto

## Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

## Scraping function definition

- Input: numbers of pages to scrap (default = 1)
- Output: list of transfers

In [1]:
def Scraping_TransferMarkt_SummerTransfers(number_pages):
    
    data = []
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    url = 'https://www.transfermarkt.com/transfers/sommertransfers/statistik?ajax=yw1&altersklasse=&ausrichtung=&land_id=0&leihe=&plus=1&sort=abloese.desc&spielerposition_id='

    for page in list(range(1, pages + 1)):

        page = url + '&page=' + str(page)
        tree = requests.get(page, headers = headers)
        soup = BeautifulSoup(tree.content, 'html.parser')
        table = soup.find('table', attrs = {'class':'items'})
        table_body = table.find('tbody')
        rows = table_body.findAll('tr')

        for row in rows[::7]:
            
            cols = row.findAll('td')
            cols = [ele.text.strip() for ele in cols]
            links = row.find_all('a', href = True)
            links = [ele['href'].split('/')[-1] for ele in links]
            countries = row.find_all('img', alt = True)[-4:]
            countries = [ele['alt'] for ele in countries]
            joinedlist = cols + links + countries
            data.append([ele for ele in joinedlist if ele]) # Get rid of empty values
        
        time.sleep(.5)
    
    return data

## Execute scraping function and export raw data to CSV file

In [29]:
df = pd.DataFrame(Scraping_TransferMarkt_SummerTransfers(324))
df.to_csv('Preprocessed_Transfers.csv', index = False)

## Import scraped data

In [6]:
df = pd.read_csv('Preprocessed_Transfers.csv')
df.head(n = 2)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,João Félix \n\n\nSecond Striker,João Félix,Second Striker,19,"70,00 mil. €",Benfica \n\n\n\n Liga NOS,Benfica,Liga NOS,Atlético Madrid \n\n\n\n LaLiga,...,294,294,PO1,13,13,ES1,SL Benfica,Portugal,Atlético Madrid,Spain
1,2,Antoine Griezmann \n\n\nCentre-Forward,Antoine Griezmann,Centre-Forward,28,"130,00 mil. €",Atlético Madrid \n\n\n\n LaLiga,Atlético Madrid,LaLiga,FC Barcelona \n\n\n\n LaLiga,...,13,13,ES1,131,131,ES1,Atlético Madrid,Spain,FC Barcelona,Spain


In [7]:
df.drop(df.columns[[1,6,9,15,18]], axis = 1, inplace = True)
df.head(n = 2)

,0,2,3,4,5,7,8,10,11,12,13,14,16,17,19,20,21,22,23
0,1,João Félix,Second Striker,19,"70,00 mil. €",Benfica,Liga NOS,Atlético Madrid,LaLiga,"126,00 mil. €",462250,294,PO1,13,ES1,SL Benfica,Portugal,Atlético Madrid,Spain
1,2,Antoine Griezmann,Centre-Forward,28,"130,00 mil. €",Atlético Madrid,LaLiga,FC Barcelona,LaLiga,"120,00 mil. €",125781,13,ES1,131,ES1,Atlético Madrid,Spain,FC Barcelona,Spain


## Define column names

In [8]:
df.columns = ['ID', 'Player', 'Position', 'Age', 'MarketValue',
              'TeamLeft', 'LeagueLeft', 'TeamJoined', 'LeagueJoined', 'TransferFee', 
              'PlayerID', 'TeamLeftID', 'LeagueLeftID', 'TeamJoinedID', 'LeagueJoinedID',
              'TeamLeft2', 'CountryLeft', 'TeamJoined2', 'CountryJoined']
df.head(n = 2)

,ID,Player,Position,Age,MarketValue,TeamLeft,LeagueLeft,TeamJoined,LeagueJoined,TransferFee,PlayerID,TeamLeftID,LeagueLeftID,TeamJoinedID,LeagueJoinedID,TeamLeft2,CountryLeft,TeamJoined2,CountryJoined
0,1,João Félix,Second Striker,19,"70,00 mil. €",Benfica,Liga NOS,Atlético Madrid,LaLiga,"126,00 mil. €",462250,294,PO1,13,ES1,SL Benfica,Portugal,Atlético Madrid,Spain
1,2,Antoine Griezmann,Centre-Forward,28,"130,00 mil. €",Atlético Madrid,LaLiga,FC Barcelona,LaLiga,"120,00 mil. €",125781,13,ES1,131,ES1,Atlético Madrid,Spain,FC Barcelona,Spain


In [9]:
df.to_csv('Precleaned_Transfers.csv', index = False)

## Data cleaning

In [10]:
#df = pd.read_csv('Precleaned_Transfers.csv')

- Remove loans

In [11]:
df = df.loc[~df['TransferFee'].str.startswith('Loan', na = False)]

- Remove invalid and unknown transfer fees

In [12]:
df = df.loc[~df['TransferFee'].str.startswith('draft', na = False)]
df = df.loc[~df['TransferFee'].str.startswith('?', na = False)]
df = df.loc[~df['TransferFee'].str.startswith('-', na = False)]

### Function definition: Conversion from string monetary to floating number

In [13]:
def currency_string_to_number(value):
   
    multiplier = 1
    number_delimiter_position = value.find(' ')
    y = value.replace(',','.')
     
    if value == 'Free transfer':
        return 0.0
    elif value[-6:] == 'mil. €':
        multiplier = 1000000
        return float(y[:number_delimiter_position])*multiplier
    elif value[-10:] == 'thousand €':
        multiplier = 1000
        return float(y[:number_delimiter_position])*multiplier
    elif value[-3:] == 'K €':
        multiplier = 1000
        return float(y[:number_delimiter_position])*multiplier
    elif value == '-':
        return 0.0
    else:
        return(float(value))

- Apply conversion function to Transfer Fees

In [14]:
df['TransferFee'] = df['TransferFee'].apply(currency_string_to_number)

- Remove invalid market value entries

In [15]:
df = df[df['MarketValue'] != 'Loros \n\n\n\n Ascenso MX Apertura']

- Apply conversion function to Market Value

In [16]:
df['MarketValue'] = df['MarketValue'].apply(currency_string_to_number)

In [17]:
# Ligas do FIFA !!!
FIFA_leagues = ['A1','AR1N','AUS1','BE1','BRA1','C1','C2','DK1',
                'ES1','ES2','FI1','FR1','FR2','GB1','GB2','GB3',
                'GB4','GR1','IT1','IT2','JAP1','KR1','L1','L2','L3',
                'MEXA','MLS1','NL1','NO1','PL1','PO1','RO1','RU1',
                'SC1','SE1','SFA1','TR1','TS1','UAE1']

In [18]:
df = df.loc[ (df['LeagueLeftID'].isin(FIFA_leagues)) & (df['LeagueJoinedID'].isin(FIFA_leagues)), : ]

In [28]:
df.Position.value_counts()

Centre-Back           239
Centre-Forward        203
Central Midfield      145
Right-Back            106
Left Winger            94
Defensive Midfield     92
Left-Back              90
Goalkeeper             88
Right Winger           84
Attacking Midfield     74
Left Midfield          23
Right Midfield         19
Second Striker         14
Name: Position, dtype: int64

In [27]:
df.LeagueJoinedID.value_counts()

IT1     113
ES1      95
FR1      93
L1       85
GB1      79
L2       74
GB2      73
RU1      53
TR1      50
BE1      48
FR2      46
RO1      32
GR1      31
NL1      30
PO1      29
PL1      29
L3       29
AUS1     27
ES2      26
DK1      25
SC1      24
AR1N     24
C1       20
IT2      19
A1       15
MEXA     14
GB3      13
TS1      12
BRA1     11
KR1      10
UAE1      9
GB4       9
C2        8
MLS1      6
SFA1      5
JAP1      4
SE1       1
Name: LeagueJoinedID, dtype: int64

In [20]:
df = df[df['Position'] != 'Defender']

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1288 entries, 0 to 2488
Data columns (total 19 columns):
ID                1288 non-null int64
Player            1288 non-null object
Position          1288 non-null object
Age               1288 non-null int64
MarketValue       1288 non-null float64
TeamLeft          1288 non-null object
LeagueLeft        1288 non-null object
TeamJoined        1288 non-null object
LeagueJoined      1288 non-null object
TransferFee       1288 non-null float64
PlayerID          1288 non-null int64
TeamLeftID        1288 non-null int64
LeagueLeftID      1288 non-null object
TeamJoinedID      1288 non-null object
LeagueJoinedID    1288 non-null object
TeamLeft2         1288 non-null object
CountryLeft       1288 non-null object
TeamJoined2       1288 non-null object
CountryJoined     1288 non-null object
dtypes: float64(2), int64(4), object(13)
memory usage: 201.2+ KB


## Drop duplicates Player entries but keep the latest transfer

In [22]:
df = df.drop_duplicates(subset = 'PlayerID', keep = 'first')

In [23]:
df.groupby('PlayerID').filter(lambda x: len(x) > 1)

,ID,Player,Position,Age,MarketValue,TeamLeft,LeagueLeft,TeamJoined,LeagueJoined,TransferFee,PlayerID,TeamLeftID,LeagueLeftID,TeamJoinedID,LeagueJoinedID,TeamLeft2,CountryLeft,TeamJoined2,CountryJoined


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1271 entries, 0 to 2488
Data columns (total 19 columns):
ID                1271 non-null int64
Player            1271 non-null object
Position          1271 non-null object
Age               1271 non-null int64
MarketValue       1271 non-null float64
TeamLeft          1271 non-null object
LeagueLeft        1271 non-null object
TeamJoined        1271 non-null object
LeagueJoined      1271 non-null object
TransferFee       1271 non-null float64
PlayerID          1271 non-null int64
TeamLeftID        1271 non-null int64
LeagueLeftID      1271 non-null object
TeamJoinedID      1271 non-null object
LeagueJoinedID    1271 non-null object
TeamLeft2         1271 non-null object
CountryLeft       1271 non-null object
TeamJoined2       1271 non-null object
CountryJoined     1271 non-null object
dtypes: float64(2), int64(4), object(13)
memory usage: 198.6+ KB


In [25]:
df

,ID,Player,Position,Age,MarketValue,TeamLeft,LeagueLeft,TeamJoined,LeagueJoined,TransferFee,PlayerID,TeamLeftID,LeagueLeftID,TeamJoinedID,LeagueJoinedID,TeamLeft2,CountryLeft,TeamJoined2,CountryJoined
0,1,João Félix,Second Striker,19,70000000.0,Benfica,Liga NOS,Atlético Madrid,LaLiga,126000000.0,462250,294,PO1,13,ES1,SL Benfica,Portugal,Atlético Madrid,Spain
1,2,Antoine Griezmann,Centre-Forward,28,130000000.0,Atlético Madrid,LaLiga,FC Barcelona,LaLiga,120000000.0,125781,13,ES1,131,ES1,Atlético Madrid,Spain,FC Barcelona,Spain
2,3,Eden Hazard,Left Winger,28,150000000.0,Chelsea,Premier League,Real Madrid,LaLiga,100000000.0,50202,631,GB1,418,ES1,Chelsea FC,England,Real Madrid,Spain
3,4,Harry Maguire,Centre-Back,26,50000000.0,Leicester,Premier League,Man Utd,Premier League,87000000.0,177907,1003,GB1,985,GB1,Leicester City,England,Manchester United,England
4,5,Matthijs de Ligt,Centre-Back,20,75000000.0,Ajax,Eredivisie,Juventus,Serie A,85500000.0,326031,610,NL1,506,IT1,Ajax Amsterdam,Netherlands,Juventus FC,Italy
5,6,Lucas Hernández,Centre-Back,23,70000000.0,Atlético Madrid,LaLiga,Bayern Munich,Bundesliga,80000000.0,281963,13,ES1,27,L1,Atlético Madrid,Spain,Bayern Munich,Germany
6,7,Nicolas Pépé,Right Winger,24,65000000.0,LOSC Lille,Ligue 1,Arsenal,Premier League,80000000.0,343052,1082,FR1,11,GB1,LOSC Lille,France,Arsenal FC,England
7,8,Frenkie de Jong,Central Midfield,22,85000000.0,Ajax,Eredivisie,FC Barcelona,LaLiga,75000000.0,326330,610,NL1,131,ES1,Ajax Amsterdam,Netherlands,FC Barcelona,Spain
8,9,Rodri,Defensive Midfield,23,80000000.0,Atlético Madrid,LaLiga,Man City,Premier League,70000000.0,357565,13,ES1,281,GB1,Atlético Madrid,Spain,Manchester City,England
9,10,João Cancelo,Right-Back,25,55000000.0,Juventus,Serie A,Man City,Premier League,65000000.0,182712,506,IT1,281,GB1,Juventus FC,Italy,Manchester City,England


## Export results to CSV file

In [26]:
#df.to_csv('Transfers.csv', index = False)